In [ ]:
from pathlib import Path

from astropy.io import fits
from astropy.visualization import simple_norm
from IPython.display import HTML
from matplotlib import animation
from matplotlib import pyplot as plt
from msvst import MSVST2D, MSVST2D1D

plt.style.use("Solarize_Light2")


def plot_image(image, title, norm=None):
    if not norm:
        norm = simple_norm(image, 'asinh', max_cut=5.2)
        
    plt.title(title)
    plt.imshow(image, origin="lower", norm=norm, cmap="gist_stern")
    plt.axis("off")
    
    return norm


def plot_frames(cube, msvst_a, msvst_b, msvst_c, norm):
    fig, ax = plt.subplots(2, 2, figsize=(5, 5))
    ax = ax.flatten()

    ims = []
    for frame, frame_a, frame_b, frame_c in zip(cube, msvst_a, msvst_b, msvst_c):
        ims_frame = []

        im = ax[0].imshow(frame, origin="lower", cmap="gist_stern", norm=norm, animated=True)
        ax[0].set_axis_off()
        ims_frame.append(im)

        im = ax[1].imshow(frame_a, origin="lower", cmap="gist_stern", norm=norm, animated=True)
        ax[1].set_axis_off()
        ims_frame.append(im)

        im = ax[2].imshow(frame_b, origin="lower", cmap="gist_stern", animated=True)
        ax[2].set_axis_off()
        ims_frame.append(im)

        im = ax[3].imshow(frame_c, origin="lower", cmap="gist_stern", animated=True)
        ax[3].set_axis_off()
        ims_frame.append(im)
        
        ims.append(ims_frame)

        plt.tight_layout()

    plt.close()

    ani = animation.ArtistAnimation(fig, ims, interval=150, blit=True, repeat_delay=1000)
    
    return ani

Using the MSVST denoising module
================================

2D MSVST
--------

In [ ]:
help(MSVST2D.denoise)

In [ ]:
image_path = Path("data", "rosseta.fits")
image = fits.getdata(image_path)

In [ ]:
output_path = image_path.parent / "rosseta_msvst_a.fits"
MSVST2D.denoise(image_path, output_path, verbose=True)
image_msvst_a = fits.getdata(output_path)

In [ ]:
output_path = image_path.with_stem("rosseta_msvst_b")
MSVST2D.denoise(image_path, output_path, kill_last=True, verbose=True)
image_msvst_b = fits.getdata(output_path)

In [ ]:
output_path = image_path.with_stem("rosseta_msvst_c")
MSVST2D.denoise(image_path, output_path, use_non_default_filter=True, verbose=True)
image_msvst_c = fits.getdata(output_path)

In [ ]:
plt.figure(figsize=(9,9))
plt.subplot(221)
norm = plot_image(image, "Original")

plt.subplot(222)
plot_image(image_msvst_a, "MSVST 2D (default)")

plt.subplot(223)
plot_image(image_msvst_b, "MSVST 2D (kill_last)")

plt.subplot(224)
plot_image(image_msvst_c, "MSVST 2D (use_non_default_filter)")

plt.tight_layout()
plt.show()

2D+1D MSVST
------------

In [ ]:
help(MSVST2D1D.denoise)

In [ ]:
cube_path = Path("data", "grid.fits")
cube = fits.getdata(cube_path)

In [ ]:
output_path = cube_path.with_stem("grid_msvst_a")
MSVST2D1D.denoise(cube_path, output_path, verbose=True)
cube_msvst_a = fits.getdata(output_path)

In [ ]:
output_path = cube_path.with_stem("grid_msvst_b")
MSVST2D1D.denoise(cube_path, output_path, kill_last=True, verbose=True)
cube_msvst_b = fits.getdata(output_path)

In [ ]:
output_path = cube_path.with_stem("grid_msvst_c")
MSVST2D1D.denoise(cube_path, output_path, use_non_default_filter=True, verbose=True)
cube_msvst_c = fits.getdata(output_path)

In [ ]:
plt.figure(figsize=(9,9))
plt.subplot(221)
norm = plot_image(cube.sum(axis=0), "Original")

plt.subplot(222)
plot_image(cube_msvst_a.sum(axis=0), "MSVST 2D+1D (default)")

plt.subplot(223)
plot_image(cube_msvst_b.sum(axis=0), "MSVST 2D+1D (kill_last)")

plt.subplot(224)
plot_image(cube_msvst_c.sum(axis=0), "MSVST 2D+1D (use_non_default_filter)")

plt.tight_layout()
plt.show()

In [ ]:
ani = plot_frames(cube, cube_msvst_a, cube_msvst_b, cube_msvst_c, norm)
HTML(ani.to_jshtml())